In [16]:
pip install  transformers torch langchain-community youtube_transcript_api

# 1. RAG - Porozmawiaj z danymi
Można użyć `document_loaders` z biblioteki `langchain-community` żeby załadować różne typy danych i "porozmawiać z nimi" używając LLMów.

Biblioteka `langchain-community` oferuje wiele różnych loaderów, między innymi:
- [Web](https://python.langchain.com/docs/integrations/document_loaders/web_base/)
- [Twitter](https://python.langchain.com/docs/integrations/document_loaders/twitter/)
- [Discord](https://python.langchain.com/docs/integrations/document_loaders/discord/)
- [Github](https://python.langchain.com/docs/integrations/document_loaders/github/)
- [CSV](https://python.langchain.com/docs/integrations/document_loaders/csv/)
- [Youtube](https://python.langchain.com/docs/integrations/document_loaders/youtube_transcript/)

i wiele więcej.

### Zaimportować Loader
Na początek trzeba zaimportować odpowiedni moduł z `langchain_community.document_loaders`. [Tutaj](https://python.langchain.com/docs/integrations/document_loaders/) można znaleźć wszystkie dostępne opcje.

Niektóre loadery wymagają dodatkowych dependencji które trzeba samodzielnie doinstalować.

In [17]:
from langchain.document_loaders import PyPDFLoader

### Załadować dane
Stworzyć loader i użyć metody `load` żeby załadować dane.

In [18]:
loader = PyPDFLoader(file_path="zaliczenie.pdf")

data = loader.load()

### Załadować Model

In [35]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

### Zaimplementować funkcję `generate`


In [42]:
def generate(prompt: str) -> str:
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.7)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [43]:
# Test the `generate` function
generate("what is the fastest car?")

"what is the fastest car? production car\nThe fastest production car is the Bugatti Chiron Super Sport 300+, with a top speed of 330 mph (531 km/h). However, it's worth noting that this car is a one-off limited edition, and it's not intended for public sale.\n\nThe fastest production car that is currently available for public sale is the Hennessey Venom F5, with a top speed of 301 mph (484 km/h). However, it's also a one-off limited edition, and it's not intended for public sale.\n\nHere are some other fast production cars:\n\n* Koenigsegg Agera RS: 273 mph (439 km/h)\n* Rimac C_Two: 258 mph (415 km/h)\n* Pagani Huayra BC: 238 mph (383 km/h)\n* SSC Tuatara: 238 mph (383 km/h)\n* Aston Martin Valkyrie: 225 mph (362 km/h)\n\nNote: The top speeds listed are manufacturer-claimed, and may not be verified by independent sources.\n\nIt's worth noting that there are other contenders for the fastest production car, such as the:\n\n* McLaren Speedtail: 225 mph (362 km/h)\n* Ferrari SF90 Stradale

### Napisać prompt
Napisać prompt bazowy na podstawie którego zostanie wygenerowana odpowiedź na `query` użytkownika w oparciu o dostępne dane.

In [44]:
BASE_PROMPT = """
You are an advanced assistant. Use the provided data to generate a precise and helpful response to the query below.

Query: {query}

Data:
{data}

Your response should be clear, concise, and directly address the query based on the data provided.
"""

### Wygenerować odpowiedź

In [45]:
query = "Help me improve this project."
prompt = BASE_PROMPT.format(query=query, data=data)

In [46]:
generate(prompt)

"\nYou are an advanced assistant. Use the provided data to generate a precise and helpful response to the query below.\n\nQuery: Help me improve this project.\n\nData:\n[Document(metadata={'source': 'zaliczenie.pdf', 'page': 0, 'page_label': '1'}, page_content='Wdrożenie innowacyjnej platformy Predictive Maintenance w \\nprzedsiębiorstwie produkcyjnym \\nMikołaj Szechniuk, Hubert Brzozowski, Kacper Miakinko \\n1. Wprowadzenie \\n1.1 Ogólna prezentacja kontekstu i problemu \\nW obliczu globalnej konkurencji i rosnących wymagań klientów \\nprzedsiębiorstwa produkcyjne potrzebują strategii pozwalających na \\nmaksymalizację dostępności maszyn oraz minimalizację kosztów napraw i \\nprzestojów. \\nTradycyjne podejście do utrzymania ruchu opiera się często na naprawach \\npoawaryjnych lub prewencyjnych w stałych odstępach czasu, co generuje \\nniepotrzebne koszty i nieprzewidywalne postoje. Rozwiązaniem jest wdrożenie \\nplatformy Predictive Maintenance, która, opierając się na danych z czuj

# 2. Prompt Chaining
Można łączyć wiele promptów jeden po drugim, aby przeprowadzać transformacje lub dodatkowe procesy na generowanych odpowiedziach przed osiągnięciem  pożądanego rezultatu.

Zadaniem będzie przekształcić zadanie programistyczne w gotowy fragment kodu, łącząc prompty w następujący łańcuch:
1. Wygenerować plan rozwiązania problemu (najlepiej w krokach)
2. Wygenerować dodatkowe kwestie, które należy wziąć pod uwagę
3. Wygenerować ostateczny kod

### Zdefiniować prompty

Prompt musi być odpowiednio sformatowany. Możesz użyć tagów HTML, markdown lub innych opcji formatowania.
W zapytaniach należy używać placeholderów.

In [47]:
GENERATE_PLAN_PROMPT = """
Given the following query:

Query: {query}

Please generate a step-by-step plan to solve the problem described in the query. Focus on clear and logical steps that will guide the resolution process. Avoid providing any code or implementation details.

Plan:
"""


In [48]:
GENERATE_CONSIDERATIONS_PROMPT = """
Based on the following query and plan:

Query: {query}
Plan:
{plan}

Please generate a list of additional considerations or constraints that should be taken into account when solving the problem. Avoid revising the plan or providing any code.

Considerations:
"""


In [49]:
GENERATE_CODE_PROMPT = """
Using the query, plan, and considerations provided below:

Query: {query}
Plan:
{plan}
Considerations:
{considerations}

Generate the final Python code that implements the solution to the problem described in the query. The code should follow the provided plan and account for the considerations. Do not include any explanations or comments outside the code.

Code:
"""


### Stwórzyć łańcuch

In [50]:
def run_chain(query: str) -> str:
    # 1. Generate a step-by-step plan
    print("Generating a step-by-step plan...")
    prompt = GENERATE_PLAN_PROMPT.format(query=query)
    plan = generate(prompt)
    print(plan)

    # 2. Generate additional considerations
    print("\n\nGenerating additional considerations...")
    prompt = GENERATE_CONSIDERATIONS_PROMPT.format(query=query, plan=plan)
    considerations = generate(prompt)
    print(considerations)

    # 3. Generate the final code snippet
    print("\n\nGenerating the final code snippet...")
    prompt = GENERATE_CODE_PROMPT.format(query=query, plan=plan, considerations=considerations)
    code = generate(prompt)
    print(code)

    return code

### Przetestować łańcuch

In [51]:
example_query_1 = "Write a Python function to find all prime numbers in a range from 1 to n."
example_query_2 = "Write a function that takes a list of words and a single word, and returns all the words in the list that are anagrams of the given word."
example_query_3 = "Whire a python program that can manage inserting into Red Black trees"

In [52]:
code_snippet = run_chain(example_query_1)

Generating a step-by-step plan...

Given the following query:

Query: Write a Python function to find all prime numbers in a range from 1 to n.

Please generate a step-by-step plan to solve the problem described in the query. Focus on clear and logical steps that will guide the resolution process. Avoid providing any code or implementation details.

Plan:
1. Understand the problem statement.
2. Define a clear and concise function signature.
3. Define the necessary variables and data structures.
4. Analyze the problem requirements and identify the key characteristics.
5. Develop a step-by-step approach to find prime numbers in a range.
6. Implement the solution in Python.
7. Test the solution with example inputs and verify its correctness.

Step-by-Step Plan:

## Step 1: Understand the Problem Statement
The problem asks to write a Python function to find all prime numbers within a specified range from 1 to n. This means the function should identify and return all numbers in the given ra

In [53]:
from IPython.display import display, Code

# Display the generated code snippet
display(Code(code_snippet, language='python'))

Using the query, plan, and considerations provided below:

Query: Write a Python function to find all prime numbers in a range from 1 to n.
Plan:

Given the following query:

Query: Write a Python function to find all prime numbers in a range from 1 to n.

Please generate a step-by-step plan to solve the problem described in the query. Focus on clear and logical steps that will guide the resolution process. Avoid providing any code or implementation details.

Plan:
1. Understand the problem statement.
2. Define a clear and concise function signature.
3. Define the necessary variables and data structures.
4. Analyze the problem requirements and identify the key characteristics.
5. Develop a step-by-step approach to find prime numbers in a range.
6. Implement the solution in Python.
7. Test the solution with example inputs and verify its correctness.

Step-by-Step Plan:

## Step 1: Understand the Problem Statement
The problem asks to write a Python function to find all prime numbers within a specified range from 1 to n. This means the function should identify and return all numbers in the given range that are only divisible by 1 and themselves.

## Step 2: Define a Clear and Concise Function Signature
The function should take two parameters: the lower and upper bounds of the range, and return a list of prime numbers within that range.

## Step 3: Define Necessary Variables and Data Structures
To solve this problem, we will need to keep track of whether each number in the range is prime or not. We can use a boolean array, where each index represents a number in the range and the value at that index indicates whether the number is prime (True) or not (False).

## Step 4
Considerations:

Based on the following query and plan:

Query: Write a Python function to find all prime numbers in a range from 1 to n.
Plan:

Given the following query:

Query: Write a Python function to find all prime numbers in a range from 1 to n.

Please generate a step-by-step plan to solve the problem described in the query. Focus on clear and logical steps that will guide the resolution process. Avoid providing any code or implementation details.

Plan:
1. Understand the problem statement.
2. Define a clear and concise function signature.
3. Define the necessary variables and data structures.
4. Analyze the problem requirements and identify the key characteristics.
5. Develop a step-by-step approach to find prime numbers in a range.
6. Implement the solution in Python.
7. Test the solution with example inputs and verify its correctness.

Step-by-Step Plan:

## Step 1: Understand the Problem Statement
The problem asks to write a Python function to find all prime numbers within a specified range from 1 to n. This means the function should identify and return all numbers in the given range that are only divisible by 1 and themselves.

## Step 2: Define a Clear and Concise Function Signature
The function should take two parameters: the lower and upper bounds of the range, and return a list of prime numbers within that range.

## Step 3: Define Necessary Variables and Data Structures
To solve this problem, we will need to keep track of whether each number in the range is prime or not. We can use a boolean array, where each index represents a number in the range and the value at that index indicates whether the number is prime (True) or not (False).

## Step 4

Please generate a list of additional considerations or constraints that should be taken into account when solving the problem. Avoid revising the plan or providing any code.

Considerations:
- The function should handle edge cases where the input range is empty or the upper bound is greater than the lower bound.
- The function should be efficient and have a time complexity of O(n log n) or better.
- The function should be able to handle large ranges.
- The function should be able to handle prime numbers greater than the maximum possible value for a number in the given range.

## Step 5

Develop a step-by

Wkleić wygenerowany kod do komórki poniżej żeby sprawdzić czy działa poprawnie.

In [56]:
def find_primes(n):
    if n < 2:
        return []

    primes = []
    for num in range(2, n + 1):
        is_prime = True
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    return primes

n = 30
print(find_primes(n))


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


# 3. Walidator tekstu - Zadanie domowe
Napisać walidator tekstu, który sprawdzi, czy tekst nie łamie żadnych reguł. Jeśli łamie, walidator powinien zwrócić odpowiednią informację.

### Zdefiniować kryteria

In [72]:
RULES = {
    "no_personal_info": "Should not contain any personal information.",
    "english_only": "Should be in English.",
    "no_questions": "Should not contain any questions.",
    # Feel free to add more rules here
}

### Zaimplementować walidator

In [73]:
VALIDATION_PROMPT = """
You are a validator. You need to ensure that the provided text meets the criteria.

<Criteria>
Code: {rule_code}
Description: {rule_description}
</Criteria>

<Text to check>
{text_to_check}
</Text to check>

# Output format
Output the result in the following JSON format:
{{
    "criteria_met": bool,  # True if the criteria is met, False otherwise
    "feedback": str  # Provide feedback if the criteria is not met, otherwise leave empty string
}}

Return just the JSON without any additional information or comments.
"""

In [89]:
import json
import re

def validate_rule(text: str, rule_code: str) -> dict:
    rule_description = RULES.get(rule_code, "No description available for this rule.")

    prompt = VALIDATION_PROMPT.format(
        rule_code=rule_code,
        rule_description=rule_description,
        text_to_check=text
    )

    response = generate(prompt)

    try:
        json_match = re.search(r"\{.*\}", response, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            result = json.loads(json_str)
            result["rule_code"] = rule_code
            return result
        else:
            raise ValueError("No JSON found in the response.")
    except json.JSONDecodeError as e:
        return {
            "criteria_met": False,
            "feedback": f"Failed to parse JSON response: {e}",
            "rule_code": rule_code
        }


### Przetestować walidator

In [88]:
def run_validator(text: str):
    for rule_code in RULES.keys():
        print(f"Checking rule '{rule_code}'...")
        result = validate_rule(text, rule_code)

        if not result["criteria_met"]:
            print(f"Rule '{rule_code}' is not met. Feedback: {result['feedback']}")
        else:
            print(f"Rule '{rule_code}' is met.")

text_to_check = "My name is John and I like to play basketball. Do you know how to play basketball?"
run_validator(text_to_check)


Checking rule 'no_personal_info'...
Rule 'no_personal_info' is not met. Feedback: Failed to parse JSON response: Expecting value: line 2 column 21 (char 22)
Checking rule 'english_only'...
Rule 'english_only' is not met. Feedback: Failed to parse JSON response: Expecting value: line 2 column 21 (char 22)
Checking rule 'no_questions'...
Rule 'no_questions' is not met. Feedback: Failed to parse JSON response: Expecting value: line 2 column 21 (char 22)


In [86]:
# text_to_check = " I like to play basketball. Do you know how to play basketball?"

In [87]:
# run_validator(text_to_check)

Checking rule 'no_personal_info'...


AssertionError: Rule 'no_personal_info' is not met. Feedback: Failed to parse JSON response: Expecting value: line 2 column 21 (char 22)

### Zaimplementować funkcję anonimizacji `anonymize` - Zadanie dodatkowe
Jeśli tekst zawiera dane które łamią powyższe reguły (np. dane osobowe), funkcja `anonymize` ma podmienić te dane na placeholder.

In [ ]:
ANONYMIZE_PROMPT = """
Your prompt here.
"""


def anonymize(text: str) -> str:
    # Implement the function that will replace the personal information with a placeholder
    # Make sure to return the anonymized text (string)
    pass

In [ ]:
print(f"Checking rule 'no_personal_info'...")
result = validate_rule(text_to_check, rule_code="no_personal_info")

if not result["criteria_met"]:
    print("Personal information found. Anonymizing the text...")
    anonymized_text = anonymize(text_to_check)
    print(anonymized_text)

    print("Re-running the validation...")
    validate_rule(anonymized_text, rule_code="no_personal_info")

    assert result["criteria_met"], "Anonymized text still contains personal information. Refine your prompt."

print("Rule is met.")